In [1]:
#  movielens_id = movielens_links[movielens_links["tmdbId"] == tmdb_id][
#     #             "movieId"
#     #         ].iloc[0]

In [2]:
import pandas as pd
links = pd.read_csv('./ml-latest-small/links.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')

df = pd.read_json("./movielens_latest-small.json").T
df["genres_list"] = df["genres"].apply(lambda x: pd.DataFrame(x)["name"].to_list() if len(x) > 0 else "")

In [3]:
df["overview"].apply(lambda x: len(x.split(" "))).describe()

count    9617.000000
mean       48.437662
std        24.788248
min         6.000000
25%        28.000000
50%        45.000000
75%        64.000000
max       177.000000
Name: overview, dtype: float64

In [4]:
links.head(1)

,movieId,imdbId,tmdbId
0,1,114709,862.0


In [5]:
ratings["tmdbId"] = ratings["movieId"].apply(lambda x: links[links["movieId"] == x]["tmdbId"].values[0].astype(int))
ratings.drop(columns=["timestamp", "movieId"], inplace=True)
ratings.head(1)

,userId,rating,tmdbId
0,1,4.0,862


In [6]:
df.head(1)

,adult,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,actors,overview_embedding,director,genres_list
862,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",102.245,1995-10-30,394400000,81,Toy Story,7.968,16849,"[{'gender': 2, 'id': 31, 'name': 'Tom Hanks', ...","[0.009772690013051, -0.045580144971609005, 0.0...",John Lasseter,"[Animation, Adventure, Family, Comedy]"


In [7]:
film_info = df[["id", "vote_average", "genres_list"]]
film_info.head(1)

,id,vote_average,genres_list
862,862,7.968,"[Animation, Adventure, Family, Comedy]"


In [8]:
m = film_info.merge(ratings, left_on="id", right_on="tmdbId")
m['mean_ML'] = m.groupby('id')['rating'].transform('mean')
m['mean_ML'] = m['mean_ML']*2
m["count_ML"] = m.groupby("id")["rating"].transform("count")
# m = m[m["count_ML"] >= 5]
m["vote_average"]= m["vote_average"].fillna(m["vote_average"].mean())
m

,id,vote_average,genres_list,userId,rating,tmdbId,mean_ML,count_ML
0,862,7.968,"[Animation, Adventure, Family, Comedy]",1,4.0,862,7.84186,215
1,862,7.968,"[Animation, Adventure, Family, Comedy]",5,4.0,862,7.84186,215
2,862,7.968,"[Animation, Adventure, Family, Comedy]",7,4.5,862,7.84186,215
3,862,7.968,"[Animation, Adventure, Family, Comedy]",15,2.5,862,7.84186,215
4,862,7.968,"[Animation, Adventure, Family, Comedy]",17,4.5,862,7.84186,215
...,...,...,...,...,...,...,...,...
100509,432131,7.658,"[Animation, Action, Comedy, Fantasy]",184,4.0,432131,8.00000,1
100510,445030,7.861,"[Animation, Action, Drama]",184,3.5,445030,7.00000,1
100511,479308,6.800,"[TV Movie, Drama]",184,3.5,479308,7.00000,1
100512,483455,8.279,"[Animation, Action, Comedy, Mystery, Crime, Fa...",184,3.5,483455,7.00000,1


In [9]:
m.describe()

,vote_average,userId,rating,tmdbId,mean_ML,count_ML
count,100514.000000,100514.000000,100514.000000,100514.000000,100514.000000,100514.000000
mean,7.025251,326.130390,3.500786,19653.889687,7.001572,58.916111
std,0.815772,182.561253,1.042339,52257.112357,1.128725,61.998967
min,0.000000,1.000000,0.500000,2.000000,1.000000,1.000000
25%,6.500000,177.000000,3.000000,711.000000,6.369427,13.000000
50%,7.087000,325.000000,3.500000,6950.000000,7.152174,39.000000
75%,7.600000,477.000000,4.000000,11593.000000,7.827957,84.000000
max,8.900000,610.000000,5.000000,525662.000000,10.000000,329.000000


In [10]:
dist = m[["rating", "genres_list"]]
dist

,rating,genres_list
0,4.0,"[Animation, Adventure, Family, Comedy]"
1,4.0,"[Animation, Adventure, Family, Comedy]"
2,4.5,"[Animation, Adventure, Family, Comedy]"
3,2.5,"[Animation, Adventure, Family, Comedy]"
4,4.5,"[Animation, Adventure, Family, Comedy]"
...,...,...
100509,4.0,"[Animation, Action, Comedy, Fantasy]"
100510,3.5,"[Animation, Action, Drama]"
100511,3.5,"[TV Movie, Drama]"
100512,3.5,"[Animation, Action, Comedy, Mystery, Crime, Fa..."


In [11]:
dist

,rating,genres_list
0,4.0,"[Animation, Adventure, Family, Comedy]"
1,4.0,"[Animation, Adventure, Family, Comedy]"
2,4.5,"[Animation, Adventure, Family, Comedy]"
3,2.5,"[Animation, Adventure, Family, Comedy]"
4,4.5,"[Animation, Adventure, Family, Comedy]"
...,...,...
100509,4.0,"[Animation, Action, Comedy, Fantasy]"
100510,3.5,"[Animation, Action, Drama]"
100511,3.5,"[TV Movie, Drama]"
100512,3.5,"[Animation, Action, Comedy, Mystery, Crime, Fa..."


In [12]:
import numpy as np
a = []
for l in dist["genres_list"].tolist():
    for g in l:
        a.append(g)

In [13]:
genres = np.unique(a)

In [14]:
genres

array(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror',
       'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'War', 'Western'], dtype='<U15')

In [15]:
all_ratings_genres = {k: [] for k in genres}

for i, r in dist.iterrows():
    for g in r["genres_list"]:
        all_ratings_genres[g].append(r["rating"])

KeyboardInterrupt: 

In [ ]:
for g in genres:
    all_ratings_genres[g] = np.mean(all_ratings_genres[g])

In [ ]:
d = pd.DataFrame(all_ratings_genres, index=["n"]).T
import scipy
d["n"] = scipy.special.softmax(d["n"])
d

,n
Action,0.046671
Adventure,0.048166
Animation,0.058188
Comedy,0.045042
Crime,0.056430
Documentary,0.068014
Drama,0.059239
Family,0.045920
Fantasy,0.047812
History,0.068039


In [ ]:
import scipy
dist =pd.DataFrame(
    { "Genre": ["comedy",  "adventure", "action", "thriller", "drama", "crime","documentary", "animated", "fantasy", "science fiction", "romance", "horror"],
        "Liked": [0.9, 0.9, 0.88, 0.84, 0.84, 0.82, 0.77, 0.7, 0.7, 0.69, 0.67, 0.52 ]
     },
)

dist["p_liked"] = scipy.special.softmax(dist["Liked"])
dist["p_disliked"] = scipy.special.softmax(1-dist["Liked"])
dist 

,Genre,Liked,p_liked,p_disliked
0,comedy,0.90,0.094416,0.072657
1,adventure,0.90,0.094416,0.072657
2,action,0.88,0.092547,0.074124
3,thriller,0.84,0.088918,0.077150
4,drama,0.84,0.088918,0.077150
5,crime,0.82,0.087157,0.078708
6,documentary,0.77,0.082907,0.082743
7,animated,0.70,0.077302,0.088743
8,fantasy,0.70,0.077302,0.088743
9,science fiction,0.69,0.076532,0.089635
